In [ ]:
import numpy
import numpy as np
import pandas as pd
import matplotlib. pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model
from keras.layers import LSTM

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
with open("Hangman Word List.txt","r") as f:
    words=f.readlines();
    words=[i[:-1] for i in words]
words = list(np.random.permutation(words))
print(len(words))


104360


In [ ]:
# During training, the model will only see words below this index.
# The remainder of the words can be used as a validation set.
train_val_split_idx = int(len(words) * 0.8)
print('Training with {} words'.format(train_val_split_idx))

MAX_NUM_INPUTS = max([len(i) for i in words[:train_val_split_idx]])
EPOCH_SIZE = train_val_split_idx
NUM_EPOCHS = 100
BATCH_SIZE = np.array([len(i) for i in words[:train_val_split_idx]]).mean()
print('Max word length: {}, average word length: {:0.1f}'.format(MAX_NUM_INPUTS, BATCH_SIZE))

Training with 83488 words
Max word length: 29, average word length: 9.1


In [ ]:
class HangmanPlayer:
    def __init__(self, word, model, lives=10):
        self.original_word = word
        self.full_word = [ord(i) - 97 for i in word]
        self.letters_guessed = set()
        self.letters_remaining = set(self.full_word)
        self.lives_left = lives
        self.obscured_words_seen = []
        self.letters_previously_guessed = []
        self.guesses = []
        self.correct_responses = []  # Stores correct responses till current guess with shape (n, 26)
        self.z = model

    def encode_obscured_word(self):
        word = [i if i in self.letters_guessed else 26 for i in self.full_word]
        obscured_word = np.zeros((len(word), 27), dtype=np.float32)
        for i, j in enumerate(word):
            obscured_word[i, j] = 1
        return obscured_word

    def encode_guess(self, guess):
        encoded_guess = np.zeros(26, dtype=np.float32)
        encoded_guess[guess] = 1
        return encoded_guess

    def encode_previous_guesses(self):
        # Create a 1 x 27 vector where 1s indicate that the letter was previously guessed
        guess = np.zeros(26, dtype=np.float32)
        for i in self.letters_guessed:
            guess[i] = 1
        return guess

    def encode_correct_responses(self):
        # To be used with cross_entropy_with_softmax, this vector must be normalized
        response = np.zeros(26, dtype=np.float32)
        for i in self.letters_remaining:
            response[i] = 1.0
        response /= response.sum()
        return response

    def store_guess_and_result(self, guess):
        # Record what the model saw as input: an obscured word and a list of previously-guessed letters
        self.obscured_words_seen.append(self.encode_obscured_word())
        self.letters_previously_guessed.append(self.encode_previous_guesses())

        # Record the letter that the model guessed, and add that guess to the list of previous guesses
        self.guesses.append(guess)
        self.letters_guessed.add(guess)

        # Store the "correct responses"
        correct_responses = self.encode_correct_responses()
        self.correct_responses.append(correct_responses)

        # Determine an appropriate reward, and reduce # of lives left if appropriate
        if guess in self.letters_remaining:
            self.letters_remaining.remove(guess)

        if self.correct_responses[-1][guess] < 0.00001:
            self.lives_left -= 1

    def run(self):
    # Play a game until we run out of lives or letters
        while self.lives_left > 0 and self.letters_remaining:
            obscured_word = self.encode_obscured_word()
            obscured_word=np.expand_dims(np.array(obscured_word),axis=0)
            previous_guesses = self.encode_previous_guesses()
            previous_guesses=np.expand_dims(previous_guesses,axis=0)
            previous_guesses=np.expand_dims(previous_guesses,axis=0)

            num_repetitions = obscured_word.shape[1]  # Extract the first dimension
            previous_guesses = np.repeat(previous_guesses, num_repetitions, axis=1)
            '''print("Shapes before prediction:")
            print("Obscured word:", obscured_word.shape)
            print("Previous guesses:", previous_guesses.shape)'''
            predictions = self.z.predict([obscured_word,previous_guesses])
            guess = np.argmax(np.squeeze(predictions[0][0]))
            self.store_guess_and_result(guess)

        # Return the observations for use in training (both inputs, predictions, and losses)
        return (np.array(self.obscured_words_seen),
                np.array(self.letters_previously_guessed),
                np.array(self.correct_responses))


    def show_words_seen(self):
        for word in self.obscured_words_seen:
            print(''.join([chr(i + 97) if i != 26 else ' ' for i in word.argmax(axis=1)]))

    def show_guesses(self):
        for guess in self.guesses:
            print(chr(guess + 97))

    def play_by_play(self):
        print('Hidden word was "{}"'.format(self.original_word))
        for i in range(len(self.guesses)):
            word_seen = ''.join([chr(i + 97) if i != 26 else ' ' for i in self.obscured_words_seen[i].argmax(axis=1)])
            print('Guessed {} after seeing "{}"'.format(chr(self.guesses[i] + 97), word_seen))

    def evaluate_performance(self):
        ended_in_success = self.lives_left > 0
        letters_in_word = set([i for i in self.original_word])
        correct_guesses = len(letters_in_word) - len(self.letters_remaining)
        incorrect_guesses = len(self.guesses) - correct_guesses
        return ended_in_success, correct_guesses, incorrect_guesses, letters_in_word


In [ ]:
z=5
p=HangmanPlayer("abc",z)
p.encode_obscured_word()
fake_guess=0
'''p.encode_obscured_word()
p.encode_previous_guesses()'''
p.store_guess_and_result(fake_guess)


In [ ]:
print(p.letters_remaining)
print(p.obscured_words_seen)


{1, 2}
[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)]


In [ ]:
p.store_guess_and_result(1)

In [ ]:
print(p.letters_remaining)
print(p.obscured_words_seen)

{2}
[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32), array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)]


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Concatenate, RepeatVector
from tensorflow.keras import Input, Model

def create_LSTM_net(input_obscured_word_seen, input_letters_guessed_previously):
    # Defining LSTM layer with return_sequences=True to get all the outputs
    lstm_output = LSTM(units=MAX_NUM_INPUTS, return_sequences=False)(input_obscured_word_seen)
    print(MAX_NUM_INPUTS)
    # Reshape the lstm_output to match the shape of input_letters_guessed_previously
    print(lstm_output.shape)
    lstm_output_reshaped = tf.expand_dims(lstm_output, axis=1)
    lstm_output_reshaped = tf.tile(lstm_output_reshaped, [1, tf.shape(input_letters_guessed_previously)[1], 1])
    # Concatenate the lstm_output_reshaped and input_letters_guessed_previously
    print(lstm_output_reshaped.shape)
    #input_letters_guessed_previously=tf.expand_dims(input_letters_guessed_previously,axis=2)
    combined_input = Concatenate()([lstm_output_reshaped, input_letters_guessed_previously])

    dense_layer = Dense(26, name='final_dense_layer')(combined_input)

    return dense_layer

input_obscured_word_seen = Input(shape=(None, 27), name='input_obscured_word_seen')
input_letters_guessed_previously = Input(shape=(None,26), name='input_letters_guessed_previously')
print(tf.shape(input_obscured_word_seen))
print(tf.shape(input_letters_guessed_previously))
z = create_LSTM_net(input_obscured_word_seen, input_letters_guessed_previously)

model = Model(inputs=[input_obscured_word_seen, input_letters_guessed_previously], outputs=z)


NotFoundError: No CPU devices are available in this process

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

# Define custom loss function to compute cross-entropy
def custom_cross_entropy(y_true, y_pred):
    return K.mean(categorical_crossentropy(y_true, y_pred))

# Define custom metric function to compute classification error
def custom_classification_error(y_true, y_pred):
    return 1.0 - K.mean(categorical_accuracy(y_true, y_pred))

# Define placeholders for correct responses
input_correct_responses = Input(shape=(26,), name='input_correct_responses')

# Define your model for training
train_model = Model(inputs=[input_obscured_word_seen, input_letters_guessed_previously],
                    outputs=z)

# Define learning rate schedule
learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Define optimizer
optimizer = SGD(learning_rate=lr_schedule, momentum=0.9)

# Compile the training model with loss and optimizer
train_model.compile(optimizer=optimizer, loss=custom_cross_entropy,run_eagerly=True)

# Now you can continue training using model.fit() function with your data


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
total_samples = 0

for epoch in range(NUM_EPOCHS):
    print(epoch)
    i = 0
    while total_samples < (epoch + 1) * EPOCH_SIZE:
        word = words[i]
        i += 1

        other_player = HangmanPlayer(word, train_model)
        words_seen, previous_letters, correct_responses = other_player.run()

        # Check if the data is not empty
        if len(words_seen) > 0:
            # Convert lists to arrays
            previous_letters = np.expand_dims(np.array(previous_letters), axis=1)
            previous_letters = np.repeat(previous_letters, words_seen.shape[1], axis=1)  # Repeat to match the length of words_seen
            correct_responses = np.expand_dims(np.array(correct_responses), axis=1)
            correct_responses = np.repeat(correct_responses, words_seen.shape[1], axis=1)

            print(words_seen.shape,previous_letters.shape)

            train_loss = train_model.fit(
                x=[words_seen, previous_letters],
                y=correct_responses,
                batch_size=int(BATCH_SIZE),  # Specify your batch size
                epochs=1,                    # Train for one epoch per minibatch
                verbose=1                    # Set verbosity level (0 = silent, 1 = progress bar, 2 = one line per epoch)
            )

            total_samples += len(words_seen)

            print(f'Step {total_samples}, Loss: {train_loss}')

    # Evaluate metrics at the end of each epoch
    eval_loss = train_model.evaluate(
        x=[words_seen, previous_letters],
        y=correct_responses
    )
    print(f'Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {eval_loss}')


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 40ms/step
(10, 4, 27) (10, 4, 26)
2/2 [==============================] - 0s 59ms/step - loss: 8.0590
Step 16799, Loss: <keras.src.callbacks.History object at 0x782092ccab90>
1/1 [==============================] - 0s 41ms/step
(10, 7, 27) (10, 7, 26)
2/2 [==============================] - 0s 64ms/step - loss: 6.4472
Step 16809, Loss: <keras.src.callbacks.History object at 0x7820929905b0>
1/1 [==============================] - 0s 44ms/step
(10, 6, 27) (10, 6, 26)
2/2 [==============================] - 0s 64ms/step - loss: 9.6709
Step 16819, Loss: <keras.src.callbacks.History object at 0x782095d85360>
1/1 [==============================] - 0s 51ms/step
(10, 10, 27) (10, 10, 26)
2/2 [==============================] - 0s 71ms/step - loss: 12.0886
Step 16829, Loss: <keras.src.callbacks.History object at 0x782095cddd20>
1/1 [==============================] - 0s 55ms/step
(10, 8, 27) (10, 8, 26)
2/2 [=